# Segmentación de consumo de Energia REDD V1.0

- Los datos contienen el consumo de energía de hogares reales, para cada casa, así como para cada circuito individual de la casa (etiquetado por el tipo principal de aparato en ese circuito).
- Los datos están destinados a ser utilizados en el desarrollo de métodos de desagregación, que pueden predecir, a partir de la señal de toda la casa qué dispositivos se están utilizando.
- La base de datos está constituida en tres partes. 
  1. low_freq / - ~ lecturas de potencia de 1Hz, toda la casa y circuitos
  2. high_freq / - formas de onda de corriente / voltaje alineadas y agrupadas
  3. high_freq_raw / - formas de onda de corriente / voltaje sin procesar

- En este estudio solo se han tenido en cuenta los correspondientes a los datos en baja frecuencia. 
- Cada archivo en baja frecuencia contiene:

 - redd / low_freq /
   - house_ {1..n} / - directorios para cada casa
   - tags.dat: etiquetas de categoría de dispositivo para cada canal
    - channel_ {1..k} .dat - lecturas de tiempo / vataje para cada canal
    
- Cada archivo channel_i.dat contiene marcas de tiempo UTC (como números enteros) y lecturas de potencia (registrando la potencia aparente del circuito) para el canal:

 1306541834   102,964  ||  UTC Unix =  1306541834  Potencia = 102,964

## Cargando librerias y datos 

In [137]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline
import time
from tensorflow import keras
from tensorflow.keras import layers
import scipy
import statsmodels
import sklearn
from datetime import datetime
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path

In [138]:
directory = os.getcwd()
pathdatos = Path(directory +'\low_freq\casas.csv')

In [139]:
pathdatos

WindowsPath('C:/Users/josel/REDD_Pruebas/low_freq/casas.csv')

In [140]:
ch2 = pd.read_csv(pathdatos,sep=r',',encoding='utf-8')

In [141]:
ch2.head(10)

,Unnamed: 0,M,d,h,m,mains_0,mains_1,mains_2,oven_3,oven_4,...,bathroom_gfi_12,electric_heat_13,stove_14,kitchen_outlets_15,kitchen_outlets_16,lighting_17,lighting_18,washer_dryer_19,washer_dryer_20,error
0,2011-04-18 13:22:13,4,18,13,22,261.0,222.20,118.83,0.0,0.0,...,1.0,0.0,0.0,2.0,0.0,65.0,46.0,0.0,0.0,0.0
1,2011-04-18 13:22:16,4,18,13,22,261.0,223.17,119.19,0.0,0.0,...,1.0,0.0,0.0,2.0,0.0,65.0,46.0,0.0,0.0,0.0
2,2011-04-18 13:22:20,4,18,13,22,262.0,223.60,118.92,0.0,0.0,...,1.0,0.0,0.0,2.0,0.0,65.0,46.0,0.0,0.0,0.0
3,2011-04-18 13:22:23,4,18,13,22,262.0,222.91,119.16,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,65.0,46.0,0.0,0.0,0.0
4,2011-04-18 13:22:26,4,18,13,22,261.0,222.94,118.83,0.0,0.0,...,1.0,0.0,0.0,2.0,0.0,65.0,46.0,0.0,0.0,0.0
5,2011-04-18 13:22:30,4,18,13,22,260.0,222.90,118.76,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,66.0,46.0,0.0,0.0,0.0
6,2011-04-18 13:22:33,4,18,13,22,260.0,222.96,118.88,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,65.0,46.0,0.0,0.0,0.0
7,2011-04-18 13:22:37,4,18,13,22,262.0,222.54,118.36,0.0,0.0,...,1.0,0.0,0.0,2.0,0.0,65.0,46.0,0.0,0.0,0.0
8,2011-04-18 13:22:40,4,18,13,22,266.0,226.03,119.17,0.0,0.0,...,1.0,0.0,0.0,2.0,0.0,65.0,46.0,0.0,0.0,0.0
9,2011-04-18 13:22:44,4,18,13,22,260.0,222.96,119.03,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,65.0,46.0,0.0,0.0,0.0


In [142]:
from datetime import datetime
ts = int("1303132933")
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

2011-04-18 13:22:13


In [143]:
directory = os.getcwd()
pathdatos = Path(directory +'\low_freq')

In [144]:
def path_house(pathdatos, house):
    """pathdatos -> (Path Object) directorio base todas casas
    house -> (int o string) Nº de casa"""
    #Concatena el ditectorio con la carpeta de cada casa
    print(pathdatos / f'house_{house}')
    return pathdatos / f'house_{house}'

# Lectura Fichero Labels con definición diferentes circuitos.-
def read_label(pathhouse):
    #Crea un diccionario de etiquetas vacio
    label = {}
    #Concatena las rutas de cada casa dada con el archivo labels.dat que 
    #contiene las etiquetas de las lecturas
    hi = pathhouse / 'labels.dat'
    #Carga la dirección general creada en hi
    with open(hi) as f:
        #Ahora lee los valores que se encuentran dentro del archivo por cada linea
        for line in f:
            #Separa las componentes de palabras en el valor line usa como separador un espacio
            splitted_line = line.split(' ')
            #Guarda las palabras encontradas usando el indice como key y el nombre como value
            label[int(splitted_line[0])] = splitted_line[1].strip() + '_' + splitted_line[0]
    #Retorna el diccionario con las etiquetas
    return label

In [145]:
path_casa = path_house(pathdatos,1)

C:\Users\josel\REDD_Pruebas\low_freq\house_1


In [146]:
labels = read_label(path_casa)

In [147]:
print(labels.values())

dict_values(['mains_1', 'mains_2', 'oven_3', 'oven_4', 'refrigerator_5', 'dishwaser_6', 'kitchen_outlets_7', 'kitchen_outlets_8', 'lighting_9', 'washer_dryer_10', 'microwave_11', 'bathroom_gfi_12', 'electric_heat_13', 'stove_14', 'kitchen_outlets_15', 'kitchen_outlets_16', 'lighting_17', 'lighting_18', 'washer_dryer_19', 'washer_dryer_20'])


In [148]:
# Función construye Dataframe datos casa seleccionada.-
def read_merge_data(pathhouse):
    #Concatena la ruta de cada casa con cada chanel_i de medida
    file = pathhouse / 'channel_1.dat'
    #Carga el diccionario de dispositivos de cada casa
    #Recuerde es un diccionario anidado label_i{{diccionario_dispositivos}}
    labels = read_label(pathhouse)
    #df es un dataframe de pandas. Se genera a partir de la separación de los datos del canal, 
    #tomando solo las filas de tiempo y de la primera medida para la red. El tiempo es 
    #un entero de 64 bits y main_1 es un flotante de 64
    df = pd.read_table(file, sep = ' ', names = ['unix_time', labels[1]], 
                                        dtype = {'unix_time': 'int64', labels[1]:'float64'}) 
    
    #Lee los directorios con la palabra channel y guarda las rutas en una variable canales tipo generador
    canales  = pathhouse.glob('channel*')   # Generador (solo puede leerse una vez!)
  
    num_apps = len(list(canales))           # Numero de circuitos en la casa
    #Bucle por cada circuito iniciando desde el segundo 2 hasta el maximo numero de circuitos
    for i in range(2, num_apps + 1):
        #File carga las direcciones de cada channel para cada casa
        file = pathhouse / f'channel_{i}.dat'
        #Carga los datos de cada archivo, con formato fecha y float, agrega encabezados
        data = pd.read_table(file, sep = ' ', names = ['unix_time', labels[i]], 
                                              dtype = {'unix_time': 'int64', labels[i]:'float64'})
        #Merge une conjuntos de datos entregados, el primer conjunto de datos es df, el cual se fabrico con 
        #los datos de mains_1, data es la llamada secuencial de los diferentes listas de datos. Inner indica que 
        #se usara como metodo un indice que servira para igualar las dos listas. On indica cual es la columna de 
        #inidces. En caso que una lista inicie con indice desfazados, elimina los datos exedentes e inicia la union 
        #desde la fila coincidente
        df = pd.merge(df, data, how = 'inner', on = 'unix_time')
        # inner=mezcla columnas comunues
        # merge = une dataframes
    #Columns es una lista de los nombres de las columnas del dataframe con todos los dispositivos en la casa
    #y el nombre de la primer columna que es el tiempo aun en utc unix
    columns = list(df.columns)

    # Vamos a añadir  siguientes variables sabemos mucha influencia en consumo:
    #    Tiempo -- Timestamp
    #    M - mes del año 
    #    h - hora del dia (por ejemplo iluminación depende del dia)
    #    D - dia del mes (en principio no debe influir mucho)
    #    m - minuto del dia
    #    d - Dia de la semana (L, M, X, J, V, S, D)  
    columns = list(df.columns)
    #Hace la transformación de UTC unix a datatime
    df['timestamp'] = df['unix_time'].astype("datetime64[s]")
    #Fracciona la fecha en mes, dia, hora, minuto y crea columnas para estos datos
    df['M'] = df['timestamp'].dt.month
    df['d'] = df['timestamp'].dt.day
    df['h'] = df['timestamp'].dt.hour
    df['m'] = df['timestamp'].dt.minute
    #Agrega las columnas timestamp, M, d, h, m. Mantiene la hora en formato Unix
    df = df[['timestamp', 'M', 'd', 'h', 'm' ] + columns]

    # mains_0 = mains_1 + mains_2
    #df['mains_0'] = df['mains_1'] + df['mains_2']

    #Crea una columa mains_0 con la suma de cada dispositivo en el mismo instante de tiempo 
    df['mains_0'] = df[columns[3:]].sum(axis=1)
    
    # ordenando las columnas de tiempo y agregando la columna mains_0 a los datos unix y medidas
    df = df[['timestamp', 'M', 'd', 'h', 'm', 'mains_0'] + columns]
    # Define la columna timestamp tiempo como indice de los datos
    df = df.set_index(df['timestamp'].values)
    # Elimina las columnas unix_time y timestamp del conjunto de datos
    df.drop(['unix_time','timestamp'], axis=1, inplace=True)
    #Retorna el nuevo conjunto de datos
    return df

In [149]:
%%time
# Cargo Labels+Datos(Dataframes) casas 1 y 2: (Tarda un poco....)
#     COLUMNAS: Labels (Circuitos)
#     FILAS: timestamp + Potencia (W) para cada label
# DFS es un Diccionario incluye Num Casa->Dataframes

dfs = {}

for i in range(1,3):
    pathcasa = path_house(pathdatos, i)
    dfs[f'labels_{i}'] = read_label(pathcasa)
    dfs[i] = read_merge_data(pathcasa)

C:\Users\josel\REDD_Pruebas\low_freq\house_1
C:\Users\josel\REDD_Pruebas\low_freq\house_2
Wall time: 11 s


In [174]:
print(f'dfs {dfs["labels_1"].keys()}')
print('*'*120)
print(f'dfs {dfs["labels_1"].values()}')

dfs dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
************************************************************************************************************************
dfs dict_values(['mains_1', 'mains_2', 'oven_3', 'oven_4', 'refrigerator_5', 'dishwaser_6', 'kitchen_outlets_7', 'kitchen_outlets_8', 'lighting_9', 'washer_dryer_10', 'microwave_11', 'bathroom_gfi_12', 'electric_heat_13', 'stove_14', 'kitchen_outlets_15', 'kitchen_outlets_16', 'lighting_17', 'lighting_18', 'washer_dryer_19', 'washer_dryer_20'])


In [163]:
print(f'dfs {dfs.keys()}')

dfs dict_keys(['labels_1', 1, 'labels_2', 2])


In [182]:
print(f'dfs {dfs["labels_1"].}')

dfs {1: 'mains_1', 2: 'mains_2', 3: 'oven_3', 4: 'oven_4', 5: 'refrigerator_5', 6: 'dishwaser_6', 7: 'kitchen_outlets_7', 8: 'kitchen_outlets_8', 9: 'lighting_9', 10: 'washer_dryer_10', 11: 'microwave_11', 12: 'bathroom_gfi_12', 13: 'electric_heat_13', 14: 'stove_14', 15: 'kitchen_outlets_15', 16: 'kitchen_outlets_16', 17: 'lighting_17', 18: 'lighting_18', 19: 'washer_dryer_19', 20: 'washer_dryer_20'}
